In [ ]:
import os
import json
import requests

In [ ]:
from os.path import join, dirname
from dotenv import load_dotenv

# current path should contain .env with 
dotenv_path = join(dirname('.'), '.env')
load_dotenv(dotenv_path)


In [ ]:
# NOTION_KEY='secret_HEcPLDZ3LKsRbt0BWivExErAcIysyQx8bxA4Z2DHtsH'

NOTION_KEY = os.getenv('NOTION_KEY')
NOTION_PAGE_ID = os.getenv('NOTION_PAGE_ID')

In [ ]:
headers = {
    "Authorization": "Bearer " + NOTION_KEY,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

In [ ]:
# def create_page(data: dict):
#     create_url = "https://api.notion.com/v1/pages"

#     payload = {"parent": {"database_id": DATABASE_ID}, "properties": data}

#     res = requests.post(create_url, headers=headers, json=payload)
#     # print(res.status_code)
#     return res

In [ ]:
search_params = {"filter": {"value": "page", "property": "object"}}
search_response = requests.post(
    f'https://api.notion.com/v1/search', 
    json=search_params, headers=headers)

(search_response.json())


In [ ]:
print(json.dumps(search_response.json(),indent=2))

In [ ]:
search_params = {"filter": {"value": "page", "property": "object"}}
search_response = requests.post(
    f'https://api.notion.com/v1/search', 
    json=search_params, headers=headers)

search_results = search_response.json()

# Check if the response contains an 'object' key
if 'object' in search_results:
    if search_results['object'] == 'list':
        # Handle the case where the token is valid but no pages are connected
        if not search_results['results']:
            print("No pages are connected to the integration.")
        else:
            # Handle the regular output
            print(json.dumps(search_results,indent=2))
    elif search_results['object'] == 'error':
        # Handle the case where the token is invalid
        if search_results.get('status') == 401 and search_results.get('code') == 'unauthorized':
            print("Error:", search_results['message'])
else:
    print("Unexpected response:", search_results)


In [ ]:
search_results = search_response.json()["results"]
page_id = search_results[0]["id"]


In [ ]:
for result in search_results:
    print(result)
    print('*******')

In [ ]:
for page in search_results:
    # Check if the page has a title
    if 'title' in page['properties']:
#     if page['properties']['title']['title']:
        title = page['properties']['title']['title'][0]['plain_text']
        page_id = page['id']
        url = page['url']
        
        print(f"Page ID: {page_id}")
        print(f"Title: {title}")
        print(f"URL: {url}")
        print("*******")
        
        # try accessing page content:
        blocks_response = requests.get(
                f"https://api.notion.com/v1/blocks/{page_id}/children", 
                headers=headers)
        blocks_data = blocks_response.json()

        # Extracting content from blocks
        for block in blocks_data['results']:
            if block['type'] == 'paragraph' and block['paragraph']['rich_text']:
                content = block['paragraph']['rich_text'][0]['plain_text']
                print(content)
        print("-------")

